# James Lab Group 4

Members: Hanlin Tian, Weizheng Wang, Ran Zhao

***After the represent the data, we transfer the column "label" to number 0 and 1. Then we tokenized the column "title" and "text" of making the iterator, then clean the data and transform them in the doc2vec. After that we split the data and put them into different set. Making a classifier of XGBoost, training the model and test it.***

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
from gensim import corpora
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import sklearn.metrics as metrics
from sklearn.model_selection import KFold

data = pd.read_csv('~/Documents/fake_news/FakeNewsTutorials/data/fake_or_real_news.csv', )
labels=data['label']
texts=data['text']
titles=data['title']
data.head()

# Load in fake_or_real_news.csv

,Unnamed: 0,title,text,label,title_vectors
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,[ 1.1533764e-02 4.2144405e-03 1.9692603e-02 ...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,[ 0.11267698 0.02518966 -0.00212591 0.021095...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,[ 0.04253004 0.04300297 0.01848392 0.048672...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,[ 0.10801624 0.11583211 0.02874823 0.061732...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,[ 1.69016439e-02 7.13498285e-03 -7.81233795e-...


## Change the labels to 1/0

In [2]:
def binarize(word):
    if word  == 'REAL':
        return 0
    else:
        return 1
Y = np.asarray(labels.apply(binarize)).reshape([labels.shape[0],])

# binarize the labels, define fake real to 0, fake to 1

In [3]:
Y.sum()/len(Y)

0.49944751381215469

## Tokenized the data

In [4]:
tokenized_title = [nltk.word_tokenize(title) for title in titles]
tokenized_text = [nltk.word_tokenize(text) for text in texts]
tokenized = tokenized_title
for i in range(titles.shape[0]):
    tokenized[i].extend(tokenized_text[i])
    
# making the iterator

## Clean the data

In [5]:
# In the first iterator, we do the clean data to delete the punctuations and other stopwords

def clean_text(tokenized_list):
    import string
    sw = nltk.corpus.stopwords.words('english')
    new_list = [[token.lower() for token in tlist if token not in string.punctuation and token.lower() not in sw] for tlist in tokenized_list]
    return new_list
cleaned = clean_text(tokenized)

## Doc2Vec

In [7]:
tagged_docs = [TaggedDocument(doc, tags=[idx]) for idx, doc in enumerate(cleaned)]
doc2vec = Doc2Vec(vector_size=300, 
                  window=5,
                  min_count=5,
                  dm = 1,
                  epochs=5)
# dm, vector_size, dm=1, window, alpha, epochs, min_count  

doc2vec.build_vocab(tagged_docs)
doc2vec.train(tagged_docs, epochs=10, total_examples=doc2vec.corpus_count)

# epochs=None, start_alpha=None

In [8]:
X = []
for i in range(titles.shape[0]):
     X.append(doc2vec.docvecs[i])

### Split the data

In [9]:
# We split the data into different dataset

validation_size = .25
test_size = .25
validation_test_size = validation_size + test_size
test_size_adjusted = test_size / validation_test_size

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y,test_size = validation_test_size)

# X_validation, X_test, Y_validation, Y_test = train_test_split(X_validation_test, y_validation_test,test_size = test_size_adjusted)

## Train the model

In [10]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [12]:
# We build the model by using the XGBoost

from sklearn import ensemble
model_xg = ensemble.GradientBoostingClassifier()
model_xg.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

## Test the model

In [13]:
Y_pred = model_xg.predict(X_test)
print(metrics.classification_report(Y_test, Y_pred, target_names=['REAL', 'FAKE']))

             precision    recall  f1-score   support

       REAL       0.92      0.89      0.90      1604
       FAKE       0.89      0.92      0.90      1564

avg / total       0.90      0.90      0.90      3168



In this prgram, we did several works:

1) Preparing the data and cleaning the data;

2) Binarizing the label and Getting the Doc2Vec presentation of the text and title;

3) Making the XGBoost classifier model;

4) Training the model and test the model.

In the first iterator of the pipeline, the dataset had been cleaned for deleting the puntuations and stopwords, also fixing all the text-cap problem. For improving the performance of the pipeline, it could be done by combining the title vectors and text vectors, making multiple dimentional vectorization to fix the model. Also, we could tune the parameter.

We used both the title and text to represent and transformed them to doc2vec representation. In the pipeline and hypertune process, learning rate is important.

Though we did not test whether the given data is balaced or not, we could use the matplotlib and seaborn for making a graph to see the balance condition of the data. We also could tune the parameter of XGBoost or build a new classifier to evaluate the performance. If the data is imbalanced, we could do the resampling, copy the minority sample when the data is not enough (oversampling) or delete the majority sample when the data is too large to keep balanced.  